In [1]:
# In this cell, some libraries were imported
import cv2
import sys
import os
from PIL import Image, ImageDraw
import pylab
import time
from matplotlib import pyplot as plt
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

In [2]:

# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)
 
# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open("Xilinx_Hackathon_2017").sheet1


###### ORB
# imports for ORB
import numpy as np
import cv2

# Initiate ORB detector
orb = cv2.ORB_create()

# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

import string
import re
####################################
######   Configurable items:  ######
####################################
# arbitary number. Fine tune once you get here
thres_hold = 250

NameError: name 'ServiceAccountCredentials' is not defined

In [ ]:
def capture(destination, image_number):
    orig_img_path = destination + str(image_number)+'.JPG'
    !fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2> /dev/null

In [ ]:
# Face Detection Function
def detectFaces(image_name):
    #print ("Face Detection Start.")
    # Read the image and convert to gray to reduce the data
    img = cv2.imread(image_name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)#Color => Gray
    
    # The haarcascades classifier is used to train data
    face_cascade = cv2.CascadeClassifier("/usr/share/opencv/haarcascades/haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)#1.3 and 5are the min and max windows of the treatures
    result = []
    for (x,y,width,height) in faces:
        result.append((x,y,x+width,y+height))
    #print ("Face Detection Complete.")
    return result

In [ ]:
def draw(image):
    img_ori = Image.open(image)
    plt.imshow(img_ori),plt.show()

In [ ]:
#Crop faces and save them in the same directory
def crop():
    filepath ="/home/xilinx/jupyter_notebooks/Final_presents/images/"
    dir_path ="/home/xilinx/jupyter_notebooks/Final_presents/"
    filecount = len(os.listdir(filepath))-1
    image_count = 1#count is the number of images
    face_cascade = cv2.CascadeClassifier("/usr/share/opencv/haarcascades/haarcascade_frontalface_default.xml")
    for fn in os.listdir(filepath): #fn 表示的是文件名
        start = time.time()
        if image_count <= filecount:
            image_name = str(image_count) + '.JPG'
            image_path = filepath + image_name
            image_new = dir_path + image_name
            #print (image_path)
            #print (image_new)
            os.system('cp '+(image_path)+ (' /home/xilinx/jupyter_notebooks/Final_presents/'))
            faces = detectFaces(image_name)
            if not faces:
                print ("There is no face!Trust me!!!")
            if faces:
                #All croped face images will be saved in a subdirectory
                face_name = image_name.split('.')[0]
                #os.mkdir(save_dir)
                count = 0
                for (x1,y1,x2,y2) in faces:
                    file_name = os.path.join(dir_path,face_name+str(count)+".jpg")
                    face_name = os.path.basename(file_name)
                    #print (face_name)
                    Image.open(image_name).crop((x1,y1,x2,y2)).save(file_name)
                    #os.system('rm -rf '+(image_path)+' /home/xilinx/jupyter_notebooks/OpenCV/Face_Detection/')
                    count+=1    
                    print("Haha, Here is one face:")
                    draw(file_name)
                    #hdmi(file_name)
                #print("The " + str(image_count) +" image were done.")
                #print("Congratulation! The total of the " + str(count) + " faces in the " +str(image_count) + " image.")
                
    

                # Integration START



                # New face from the webcam
                image = file_name

                # Find the Key point
                img = cv2.imread(image,0)          # queryImage

                # find the keypoints and descriptors with ORB
                kp1, des1 = orb.detectAndCompute(img,None)

                # Extract and print all of the values
                all_faces_str = sheet.get_all_values()


                # convert google doc strings to numbers
                data = all_faces_str[1:]
                face_order = 0
                for row in data:

                    face_order += 1
                    myTotal = np.array([])
                    for element in row:
                        myArray = np.array(eval(element))
                        myTotal = np.vstack([myTotal,myArray]) if myTotal.size else myArray


                    # convert data type
                    myTotal_uint8 = np.uint8(myTotal) 

                    # Add each image descriptor list from the database
                    clusters = np.array([myTotal_uint8])
                    bf.add(clusters)

                    # Train: Does nothing for BruteForceMatcher though.
                    bf.train()

                    matches = bf.match(des1,myTotal_uint8)
                    matches = sorted(matches, key = lambda x:x.distance)

                    facest_face = 0
                    numb_matches = (len(matches))
                    if numb_matches > facest_face:
                        facest_face = numb_matches

                    face_found = False
                    if facest_face > thres_hold:
                        face_found = True
                        found_face_order = face_order



                # Add face to database if not found, otherwise report the face
                if face_found == True:
                    print ("FACE FOUND! Face number:")
                    print (found_face_order)
                else:
                    print ("Face NOT found. adding face to database")
                    sheet.append_row('')
                    col_in = 1
                    row_in = sheet.row_count

                    collumn_list = []
                    des_cnt = 0
                    des_per_cell = 100
                    for collumn in des1:

                        collumn_list.append(collumn.tolist())
                        # Append des_per_cell # of collumns into a single collumn and upload
                        if des_cnt >= des_per_cell -1:
                            print ("cloud")
                            sheet.update_cell(row_in, col_in, collumn_list)
                            collumn_list = []
                            col_in += 1
                            des_cnt = 0
                        else:
                            des_cnt += 1



                # Integration END
            
            
           
                
                os.system('rm -rf '+(image_path))
            os.system('rm -rf '+(image_new))
        end = time.time()
        TimeSpan = end - start
        if image_count <= filecount:
            print ("The time of " + str(image_count) + " image is " +str(TimeSpan) + " s.")
        image_count = image_count + 1    

In [ ]:
def hdmi(file_name):
    base = BaseOverlay("base.bit")
    # monitor configuration: 640*480 @ 60Hz
    Mode = VideoMode(640,480,24)
    hdmi_out = base.video.hdmi_out
    hdmi_out.configure(Mode,PIXEL_BGR)
    hdmi_out.start()
    
    # monitor (output) frame buffer size
    frame_out_w = 1920
    frame_out_h = 1080
    # camera (input) configuration
    frame_in_w = 67
    frame_in_h = 67
    
    import cv2

    videoIn = cv2.VideoCapture(file_name)
    videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
    videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

    print("Capture device is open: " + str(videoIn.isOpened()))
    
    # Capture webcam image
    import numpy as np

    ret, frame_vga = videoIn.read()

    # Display webcam image via HDMI Out
    if (ret):      
        outframe = hdmi_out.newframe()
        outframe[0:67,0:67,:] = frame_vga[0:67,0:67,:]
        hdmi_out.writeframe(outframe)
    else:
        raise RuntimeError("Failed to read from camera.")

In [ ]:
def main():
    while(base.buttons[0].read()==0):
        image_number = 1
        filepath ="/home/xilinx/jupyter_notebooks/Final_presents/images/"
        capture(filepath, image_number)
        crop()

In [ ]:
main()